# ロジスティック回帰の実装

このノートブックでは、ロジスティック回帰をスクラッチから実装し、scikit-learnとの比較を行います。

## 学習目標
- ロジスティック回帰の理論的理解
- シグモイド関数の実装
- 交差エントロピー損失の計算
- 勾配降下法による最適化
- 決定境界の可視化


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. データの準備

まず、分類用のデータセットを生成します。


In [ ]:
# データセットの生成
X, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    random_state=42
)

# データの可視化
plt.figure(figsize=(10, 6))
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='red', label='Class 0', alpha=0.7)
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='blue', label='Class 1', alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Generated Classification Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"データセットの形状: {X.shape}")
print(f"クラスの分布: {np.bincount(y)}")


In [ ]:
# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"訓練データの形状: {X_train_scaled.shape}")
print(f"テストデータの形状: {X_test_scaled.shape}")
print(f"訓練データの平均: {X_train_scaled.mean(axis=0)}")
print(f"訓練データの標準偏差: {X_train_scaled.std(axis=0)}")


## 2. ロジスティック回帰の実装

### 2.1 シグモイド関数の実装


In [ ]:
def sigmoid(z):
    """
    シグモイド関数の実装
    
    Parameters:
    z: 線形結合の結果
    
    Returns:
    sigmoid(z): シグモイド関数の値
    """
    # 数値的安定性のため、zが大きすぎる場合は制限
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

# シグモイド関数の可視化
z = np.linspace(-10, 10, 100)
sigmoid_values = sigmoid(z)

plt.figure(figsize=(10, 6))
plt.plot(z, sigmoid_values, 'b-', linewidth=2, label='Sigmoid Function')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Decision Threshold (0.5)')
plt.axvline(x=0, color='g', linestyle='--', alpha=0.7, label='z = 0')
plt.xlabel('z')
plt.ylabel('σ(z)')
plt.title('Sigmoid Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("シグモイド関数の特徴:")
print(f"σ(0) = {sigmoid(0):.3f}")
print(f"σ(5) = {sigmoid(5):.3f}")
print(f"σ(-5) = {sigmoid(-5):.3f}")


### 2.2 交差エントロピー損失の実装


In [ ]:
def cross_entropy_loss(y_true, y_pred):
    """
    交差エントロピー損失の計算
    
    Parameters:
    y_true: 実際のラベル
    y_pred: 予測確率
    
    Returns:
    loss: 交差エントロピー損失
    """
    # 数値的安定性のため、予測値をクリップ
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    
    # 交差エントロピー損失の計算
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

# 損失関数の可視化
y_true_example = np.array([1, 0, 1, 0])
y_pred_example = np.linspace(0.01, 0.99, 100)

losses = []
for pred in y_pred_example:
    loss = cross_entropy_loss(y_true_example, np.full_like(y_true_example, pred))
    losses.append(loss)

plt.figure(figsize=(10, 6))
plt.plot(y_pred_example, losses, 'b-', linewidth=2)
plt.xlabel('Predicted Probability')
plt.ylabel('Cross-Entropy Loss')
plt.title('Cross-Entropy Loss vs Predicted Probability')
plt.grid(True, alpha=0.3)
plt.show()

print("交差エントロピー損失の特徴:")
print(f"y_true=1, y_pred=0.9 の損失: {cross_entropy_loss([1], [0.9]):.3f}")
print(f"y_true=1, y_pred=0.1 の損失: {cross_entropy_loss([1], [0.1]):.3f}")
print(f"y_true=0, y_pred=0.1 の損失: {cross_entropy_loss([0], [0.1]):.3f}")
print(f"y_true=0, y_pred=0.9 の損失: {cross_entropy_loss([0], [0.9]):.3f}")


### 2.3 ロジスティック回帰クラスの実装


In [ ]:
class LogisticRegressionScratch:
    """
    ロジスティック回帰のスクラッチ実装
    """
    
    def __init__(self, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
        """
        パラメータの初期化
        
        Parameters:
        learning_rate: 学習率
        max_iterations: 最大反復回数
        tolerance: 収束判定の閾値
        """
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.cost_history = []
    
    def fit(self, X, y):
        """
        モデルの訓練
        
        Parameters:
        X: 特徴量
        y: ラベル
        """
        # パラメータの初期化
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # 勾配降下法
        for i in range(self.max_iterations):
            # 線形結合の計算
            z = np.dot(X, self.weights) + self.bias
            
            # シグモイド関数の適用
            predictions = sigmoid(z)
            
            # 損失の計算
            cost = cross_entropy_loss(y, predictions)
            self.cost_history.append(cost)
            
            # 勾配の計算
            dw = (1 / n_samples) * np.dot(X.T, (predictions - y))
            db = (1 / n_samples) * np.sum(predictions - y)
            
            # パラメータの更新
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
            # 収束判定
            if i > 0 and abs(self.cost_history[-1] - self.cost_history[-2]) < self.tolerance:
                print(f"収束しました。反復回数: {i+1}")
                break
        
        return self
    
    def predict_proba(self, X):
        """
        確率の予測
        
        Parameters:
        X: 特徴量
        
        Returns:
        probabilities: 予測確率
        """
        z = np.dot(X, self.weights) + self.bias
        return sigmoid(z)
    
    def predict(self, X, threshold=0.5):
        """
        クラスの予測
        
        Parameters:
        X: 特徴量
        threshold: 決定閾値
        
        Returns:
        predictions: 予測クラス
        """
        probabilities = self.predict_proba(X)
        return (probabilities >= threshold).astype(int)
    
    def score(self, X, y):
        """
        精度の計算
        
        Parameters:
        X: 特徴量
        y: ラベル
        
        Returns:
        accuracy: 精度
        """
        predictions = self.predict(X)
        return accuracy_score(y, predictions)

# モデルの訓練
model = LogisticRegressionScratch(learning_rate=0.1, max_iterations=1000)
model.fit(X_train_scaled, y_train)

# 損失の履歴を可視化
plt.figure(figsize=(10, 6))
plt.plot(model.cost_history)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Cost History During Training')
plt.grid(True, alpha=0.3)
plt.show()

print(f"最終的な損失: {model.cost_history[-1]:.4f}")
print(f"重み: {model.weights}")
print(f"バイアス: {model.bias:.4f}")


## 3. 決定境界の可視化


In [ ]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """
    決定境界の可視化
    
    Parameters:
    model: 訓練済みモデル
    X: 特徴量
    y: ラベル
    title: グラフのタイトル
    """
    # メッシュグリッドの作成
    h = 0.01
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # 予測
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # プロット
    plt.figure(figsize=(12, 8))
    plt.contourf(xx, yy, Z, levels=50, alpha=0.8, cmap='RdYlBu')
    plt.colorbar(label='Predicted Probability')
    
    # データポイントのプロット
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', linewidth=1)
    plt.colorbar(scatter, label='True Class')
    
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.show()

# 決定境界の可視化
plot_decision_boundary(model, X_train_scaled, y_train, "Training Data - Decision Boundary")


## 4. モデルの評価


In [ ]:
# 予測の実行
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)
y_pred_proba_test = model.predict_proba(X_test_scaled)

# 精度の計算
train_accuracy = model.score(X_train_scaled, y_train)
test_accuracy = model.score(X_test_scaled, y_test)

print("=== モデルの評価結果 ===")
print(f"訓練データの精度: {train_accuracy:.4f}")
print(f"テストデータの精度: {test_accuracy:.4f}")

# 混同行列の表示
print("\n=== 混同行列（テストデータ） ===")
cm = confusion_matrix(y_test, y_pred_test)
print(cm)

# 分類レポートの表示
print("\n=== 分類レポート（テストデータ） ===")
print(classification_report(y_test, y_pred_test))

# 予測確率の分布
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(y_pred_proba_test[y_test == 0], bins=20, alpha=0.7, label='Class 0', color='red')
plt.hist(y_pred_proba_test[y_test == 1], bins=20, alpha=0.7, label='Class 1', color='blue')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(y_pred_proba_test, y_test, alpha=0.6)
plt.xlabel('Predicted Probability')
plt.ylabel('True Label')
plt.title('Predicted Probability vs True Label')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. scikit-learnとの比較


In [ ]:
# scikit-learnのロジスティック回帰
sklearn_model = LogisticRegression(random_state=42)
sklearn_model.fit(X_train_scaled, y_train)

# 予測
sklearn_pred_test = sklearn_model.predict(X_test_scaled)
sklearn_proba_test = sklearn_model.predict_proba(X_test_scaled)[:, 1]

# 精度の比較
sklearn_accuracy = sklearn_model.score(X_test_scaled, y_test)

print("=== scikit-learnとの比較 ===")
print(f"自作モデルの精度: {test_accuracy:.4f}")
print(f"scikit-learnの精度: {sklearn_accuracy:.4f}")
print(f"精度の差: {abs(test_accuracy - sklearn_accuracy):.4f}")

# 係数の比較
print(f"\n=== 係数の比較 ===")
print(f"自作モデルの重み: {model.weights}")
print(f"scikit-learnの重み: {sklearn_model.coef_[0]}")
print(f"自作モデルのバイアス: {model.bias:.4f}")
print(f"scikit-learnのバイアス: {sklearn_model.intercept_[0]:.4f}")

# 予測確率の比較
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(model.predict_proba(X_test_scaled), sklearn_proba_test, alpha=0.6)
plt.plot([0, 1], [0, 1], 'r--', alpha=0.8)
plt.xlabel('自作モデルの予測確率')
plt.ylabel('scikit-learnの予測確率')
plt.title('予測確率の比較')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(model.predict_proba(X_test_scaled), sklearn_proba_test, alpha=0.6)
plt.plot([0, 1], [0, 1], 'r--', alpha=0.8)
plt.xlabel('自作モデルの予測確率')
plt.ylabel('scikit-learnの予測確率')
plt.title('予測確率の比較（拡大）')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 6. 演習問題

### 演習1: 学習率の影響
異なる学習率でモデルを訓練し、収束の様子を比較してみましょう。

### 演習2: 正則化の実装
L2正則化を追加して、過学習を防ぐ実装を作成してみましょう。

### 演習3: 決定閾値の調整
決定閾値を変更して、PrecisionとRecallのトレードオフを確認してみましょう。


## まとめ

このノートブックでは、ロジスティック回帰をスクラッチから実装しました。

**学習した内容**：
- シグモイド関数の実装と可視化
- 交差エントロピー損失の計算
- 勾配降下法による最適化
- 決定境界の可視化
- scikit-learnとの比較

**重要なポイント**：
- ロジスティック回帰は確率的な出力を提供
- 線形決定境界を持つ
- 数値的安定性に注意が必要
- 特徴量の標準化が重要

**次のステップ**：
- 多クラス分類への拡張
- 正則化の実装
- より高度な最適化手法の学習
